## Health, Wealth of Nations from 1800-2008

In [1]:
import os
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
from bqplot import Figure, Tooltip, Label
from bqplot import Axis, ColorAxis
from bqplot import LogScale, LinearScale, OrdinalColorScale
from bqplot import Scatter, Lines
from bqplot import CATEGORY10

In [3]:
from ipywidgets import HBox, VBox, IntSlider, Play, jslink

In [4]:
from more_itertools import flatten

---
### Get Data

In [34]:
year_start = 1800

df = pd.read_json("data_files/nations.json")
df.head()

,income,lifeExpectancy,name,population,region
0,"[[1800, 359.93], [1820, 359.93], [1913, 556.12...","[[1800, 26.98], [1940, 26.98], [1950, 29.22], ...",Angola,"[[1800, 1567028], [1820, 1567028], [1940, 3738...",Sub-Saharan Africa
1,"[[1800, 553.72], [1820, 553.72], [1913, 855.53...","[[1800, 31], [1944, 31], [1950, 36.53], [1951,...",Benin,"[[1800, 636559], [1820, 636559], [1950, 167266...",Sub-Saharan Africa
2,"[[1800, 407.36], [1820, 407.36], [1913, 629.4]...","[[1800, 33.6], [1945, 33.6], [1950, 46.82], [1...",Botswana,"[[1800, 121000], [1904, 121000], [1911, 125000...",Sub-Saharan Africa
3,"[[1800, 454.33], [1820, 454.33], [1913, 497.44...","[[1800, 29.2], [1945, 29.2], [1950, 32.89], [1...",Burkina Faso,"[[1800, 1665421], [1820, 1665421], [1950, 4376...",Sub-Saharan Africa
4,"[[1800, 447.59], [1820, 447.59], [1913, 353.82...","[[1800, 31.5], [1945, 31.5], [1950, 38.42], [1...",Burundi,"[[1800, 899097], [1820, 899097], [1950, 236252...",Sub-Saharan Africa


In [36]:
list_rows_to_drop = \
(df['income']
 .apply(len)
 .where(lambda i: i < 10)
 .dropna()
 .index
 .tolist()
)

df.drop(list_rows_to_drop, inplace=True)

In [37]:
dict_dfs = {}

for COL in ['income', 'lifeExpectancy', 'population']:
    df1 = \
    DataFrame(df
              .loc[:, COL]
              .map(lambda l: (DataFrame(l)
                              .set_index(0)
                              .squeeze()
                              .reindex(range(1800, 2009))
                              .interpolate()
                              .to_dict()))
              .tolist())
    df1.index = df.name
    dict_dfs[COL] = df1

In [ ]:
def get_data(year):
    """
    """
    income = dict_dfs['income'].loc[:, year]
    lifeExpectancy = dict_dfs['lifeExpectancy'].loc[:, year]
    population = dict_dfs['population'].loc[:, year]
    return income, lifeExpectancy, population

get_min_max_from_df = lambda df: (df.min().min(), df.max().max())

---
### Create Tooltip

In [10]:
tt = Tooltip(fields=['name', 'x', 'y'], 
             labels=['Country', 'IncomePerCapita', 'LifeExpectancy'])

---
### Create Scales

In [11]:
# Income 
income_min, income_max = get_min_max_from_df(dict_dfs['income'])
x_sc = LogScale(min=income_min, 
                max=income_max)

# Life Expectancy 
life_exp_min, life_exp_max = get_min_max_from_df(dict_dfs['lifeExpectancy'])
y_sc = LinearScale(min=life_exp_min, 
                   max=life_exp_max)

# Population
pop_min, pop_max = get_min_max_from_df(dict_dfs['population'])
size_sc = LinearScale(min=pop_min, 
                      max=pop_max)

# Color
c_sc = OrdinalColorScale(domain=df['region'].unique().tolist(), 
                         colors=CATEGORY10[:6])

---
### Create Axes

In [12]:
ax_y = Axis(label='Life Expectancy', 
            scale=y_sc, 
            orientation='vertical', 
            side='left', 
            grid_lines='solid')

ax_x = Axis(label='Income per Capita', 
            scale=x_sc, 
            grid_lines='solid')

---
## Create Marks

### 1. Scatter

In [13]:
cap_income, life_exp, pop = get_data(year_start)

In [14]:
scatter_ = Scatter(x=cap_income, 
                   y=life_exp, 
                   color=df['region'], 
                   size=pop,
                   names=df['name'], 
                   display_names=False,
                   scales={
                       'x': x_sc, 
                       'y': y_sc, 
                       'color': c_sc, 
                       'size': size_sc
                   },
                   default_size=4112, 
                   tooltip=tt, 
                   animate=True, 
                   stroke='Black',
                   unhovered_style={'opacity': 0.5})

### 2. Line

In [15]:
line_ = Lines(x=dict_dfs['income'].loc['Angola'].values, 
              y=dict_dfs['lifeExpectancy'].loc['Angola'].values, 
              colors=['Gray'],
              scales={
                  'x': x_sc, 
                  'y': y_sc
              }, 
              visible=False)

---
### Create Label

In [16]:
year_label = Label(x=[0.75], 
                   y=[0.10],
                   font_size=50, 
                   font_weight='bolder', 
                   colors=['orange'],
                   text=[str(year_start)],
                   enable_move=True)

---
## Construct the Figure

In [39]:
time_interval = 10

fig_ = \
Figure(
    marks=[scatter_, line_, year_label], 
    axes=[ax_x, ax_y],
    title='Health and Wealth of Nations', 
    animation_duration=time_interval
)

fig_.layout.min_width = '960px'
fig_.layout.min_height = '640px'

fig_

Figure(animation_duration=10, axes=[Axis(label='Income per Capita', scale=LogScale(max=119849.29, min=281.91),…

---
## Add Interactivity

- Update chart when year changes

In [40]:
slider_ = IntSlider(
    min=year_start, 
    max=2008, 
    step=1, 
    description='Year: ', 
    value=year_start)

def on_change_year(change):
    """
    """
    scatter_.x, scatter_.y, scatter_.size = get_data(slider_.value)
    year_label.text = [str(slider_.value)]

slider_.observe(on_change_year, 'value')

slider_

IntSlider(value=1800, description='Year: ', max=2008, min=1800)

- Display line when hovered

In [41]:
def on_hover(change):
    """
    """
    if change.new is not None:
        display(change.new)
        line_.x = dict_dfs['income'].iloc[change.new + 1]
        line_.y = dict_dfs['lifeExpectancy'].iloc[change.new + 1]
        line_.visible = True
    else:
        line_.visible = False

In [42]:
scatter_.observe(on_hover, 'hovered_point')

---
## Add Animation!

In [43]:
play_button = Play(
    min=1800, 
    max=2008, 
    interval=time_interval
)

jslink(
    (play_button, 'value'), 
    (slider_, 'value')
)

---
## Create the GUI

In [44]:
VBox([play_button, slider_, fig_])